In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import numpy as np
import time

post_list = []
for page in range(1, 3):
    response = requests.get(f"https://br.tradingview.com/ideas/page-{page}/") 
    soup = BeautifulSoup(response.text, 'html.parser')
    posts = soup.find_all("div", class_="tv-feed__item")

    for post in posts:
        try:
            update = post.find(class_='tv-card-stats__time').text
            likes = post.find(class_="tv-card-social-item__count").text
            timestamp_span_list = post.find_all("span", class_='tv-card-stats__time')
            timestamp = None
            for span in timestamp_span_list:
                if span.attrs.get('data-timestamp') is not None:
                    timestamp = span.attrs.get('data-timestamp')
                    break
            if timestamp is None:
              continue
            dt_object = datetime.datetime.fromtimestamp(int(float(timestamp)))
            ticker = post.find(class_="tv-widget-idea__symbol-info").find("a").text
            trader = post.find(class_="tv-card-user-info__name").text
            profile_url = post.find(class_="tv-card-user-info__main-wrap js-userlink-popup").attrs.get('href')

            recommendation_list = post.find_all("span", class_="content-PlSmolIm badge-idea-content-ZleujXqe")
            try:
              recommendation = recommendation_list[0].text
            except IndexError:
              recommendation = "N/A"

            title = post.find(class_="tv-widget-idea__title-row").find("a").text
            #description = post.find(class_="tv-widget-idea__description-row tv-widget-idea__description-row--clamped js-widget-idea__popup").text
            #href = post.find(class_="tv-widget-idea__cover-link js-widget-idea__popup").attrs.get('data-href')

            href = post.find(class_="tv-widget-idea__cover-link js-widget-idea__popup").attrs.get('data-href')
            response2 = requests.get(f"https://www.tradingview.com{href}")
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            post2 = soup2.find("div", class_="tv-chart-view__description-wrap js-chart-view__description")
            content_elements = post2.find_all(class_='tv-chart-view__description selectable') #pega os não ativos e comentários e os comentários dos ativos
            content_elements1 = post2.find_all(class_='tv-chart-updates__description tv-chart-view__description selectable') #pega os ativos
            content_completo = ''.join([c.get_text() for c in content_elements if c.get_text().strip()])
            #tags =  post2.find_all(class_='text-OxpkAz7E').text
            #tags1 = ''.join([c.get_text() for c in tags if c.get_text().strip()])
            #print(tags1)
            #print(content_completo)
            content_completo1 = ''.join([c.get_text() for c in content_elements1 if c.get_text().strip()])

            href2 = post.find(class_="tv-card-user-info__main-wrap js-userlink-popup").attrs.get('data-username')
            response3 = requests.get(f"https://www.tradingview.com/u/{href2}")

            soup3 = BeautifulSoup(response3.text, 'html.parser')
            #post3 = soup3.find("div", class_="tv-profile__socials")
            #seguidores = post3.find(class_='tv-profile__social-item-value').text

            ideas = soup3.find("div", class_="tv-profile__socials")
            if ideas:
              ideas_element = ideas.find(class_='tv-profile__social-item-value').text

            time.sleep(5)

            post_item = {
            #'Tags': tags,
            'Update': update,
            'Date': dt_object,
            'Ticker': ticker,
            'Likes':likes,
            'Trader': trader,
            'Seguidores': ideas_element,
            'Profile_Url':profile_url,
            'Recommendation': recommendation,
            'Title': title,
            #'Description': description,
            #'Href': href,
            #'Data': date,
            'Href': href,
            'X':content_completo,
            'Y':content_completo1,
            }
            post_list.append(post_item)
        except AttributeError:
            print(f"Erro ao parsear: {post.find('title')}")


data = pd.DataFrame(post_list)
#print(df.head())
data.to_csv('tradingview-ideas.csv', index=False)

Erro ao parsear: None
Erro ao parsear: None
Erro ao parsear: None


In [2]:
data['Content'] = data.apply(lambda row: str(row['X']) + ' ' + str(row['Y']), axis=1)
data = data.drop(['Href','X', 'Y'], axis=1)

recommendation_copy = np.array(data['Recommendation'])
data['Sentiment'] = recommendation_copy

data['Sentiment'].replace('Viés de Alta', '1', inplace=True)  #Long = 1
data['Sentiment'].replace('Viés de Baixa', '-1', inplace=True) #Short = -1
data['Sentiment'].replace('N/A', '0', inplace=True) #Neutro = 0

data.to_csv('database-tradingview.csv', index=False)
display(data)

,Update,Date,Ticker,Likes,Trader,Seguidores,Profile_Url,Recommendation,Title,Content,Sentiment
0,,2023-10-30 08:44:46,BTCUSD,32,GustavoCorrea1,2956,/u/GustavoCorrea1/,Viés de alta,PANORAMA BITCOIN 30/10 - SEMANAL EXTREMAMENTE ...,Olá Amigos! Fica aqui minha análise do Bitcoin...,Viés de alta
1,,2023-10-30 15:57:45,USDJPY,1,jonis32,1,/u/jonis32/,Viés de baixa,Banco central do Japão (BOJ) irá intervir na m...,Como pode ser observado nas marcações feitas n...,Viés de baixa
2,,2023-10-30 18:38:32,EWZ,1,lucascmoraes,93,/u/lucascmoraes/,N/A,Observatório - Análise do Panorama Nacional - ...,Nesse vídeo destaco os principais pontos e a e...,0
3,,2023-10-30 08:36:55,BTCUSD,6,mgptec,56,/u/mgptec/,N/A,Resumo do Mercado 30/10/2023,1 - Indices começam semana se recuperando de r...,0
4,,2023-10-30 09:02:18,BTCUSDT,7,MarceloTorres,3172,/u/MarceloTorres/,N/A,FOCO NOS ÍNDICES: BTC pode ir a 36k e 4af. pod...,Fala Trader! Segue a minha opinião em mais um...,0
5,Atualizado,2023-10-29 23:35:53,BTCUSDT,4,investida01,114,/u/investida01/,N/A,"E agora, Bitcoin? Euforia ou Balde de Água Fria?",\n\n\n Comment: ...,0
6,,2023-10-30 16:40:24,USDCHF,2,BlackBull_Markets,114,/broker/BlackBull_Markets/,N/A,Franco Suíço sobe no voo dos investidores para...,Franco Suíço sobe no voo dos investidores para...,0
7,,2023-10-30 17:38:56,ABEV3,1,EvaldoInvest,20,/u/EvaldoInvest/,Viés de alta,"""AMBEV-#ABEV3 PODE BUSCAR REPIQUE NOS 12,86""",O ativo está tem tendência de baixa desde a di...,Viés de alta
8,,2023-10-30 13:08:31,EURUSD,1,NatanaelB_Costa,0,/u/NatanaelB_Costa/,Viés de baixa,Análise no EURUSD 60m,Análise baseada no tempo gráfico de 60m preço ...,Viés de baixa
9,,2023-10-30 23:35:02,BTCUSDT,1,Rafael_Lagosta,106,/u/Rafael_Lagosta/,Viés de baixa,"""Ivagina você se depentelho o mundo se descaba...",O Bitcoin tem sido uma das maiores sensações f...,Viés de baixa
